In [73]:
#-*-coding:utf-8-*-
import urllib.request
import json

openweather_api_url = "https://api.openweathermap.org/data/2.5/"
service_key = "1ec716427f7679f0fe76cb6036c13ba0"


def getNowCity(city) :
    global openweather_api_url, service_key
    global c_temp

    # API 요청시 필요한 인수값 정의
    ow_api_url = openweather_api_url + "weather"
    payload = "?q=" + city + "&" + "appid=" + service_key #"lang=kr" 옵션을 추가하면 날씨설명을 한글로 받을 수 있음.
    url_total = ow_api_url + payload

    # API 요청하여 데이터 받기
    req = urllib.request.urlopen(url_total)
    res = req.readline()

    # 받은 값 JSON 형태로 정제하여 반환
    items = json.loads(res)
    
    weather_main = items['weather'][0]['main']
    c_temp = int(items['main']['temp'])-273.15
    
    return weather_main


getNowCity("Seoul")


'Clouds'

In [76]:
import random

def get_food_list(current_weather):
    rain = "파전, 전집, 삼계탕, 찜닭, 철판, 곱창, 중식, 오리, 뷔페".split(',')
    snow = "삼겹살, 이태리, 설렁탕, 곰탕, 동남아, 패밀리".split(',')

    if current_weather in ['Rain', 'Thunderstorm', 'Drizzle']:
        food_list = random.sample(rain, 3)
    elif current_weather == 'Snow':
        food_list = random.sample(snow, 3)
    else:
        food_list = None

    return food_list

In [71]:
print(get_food_list(getNowCity("Seoul")))
print(get_food_list("Rain"))

None
[' 삼계탕', ' 철판', ' 뷔페']


In [ ]:
import pandas as pd
import folium
import urllib.request
import json
import random
from folium import IFrame

openweather_api_url = "https://api.openweathermap.org/data/2.5/"
service_key = "1ec716427f7679f0fe76cb6036c13ba0"

def getNowCity(city):
    global openweather_api_url, service_key
    global c_temp

    ow_api_url = openweather_api_url + "weather"
    payload = "?q=" + city + "&" + "appid=" + service_key
    url_total = ow_api_url + payload

    req = urllib.request.urlopen(url_total)
    res = req.readline()

    items = json.loads(res)

    weather_main = items['weather'][0]['main']
    c_temp = int(items['main']['temp']) - 273.15

    return weather_main

def get_food_list(current_weather):
    rain = "파전, 전집, 삼계탕, 찜닭, 철판, 곱창, 중식, 오리, 뷔페".split(',')
    snow = "삼겹살, 이태리, 설렁탕, 곰탕, 동남아, 패밀리".split(',')
    if current_weather in ['Rain', 'Thunderstorm', 'Drizzle']:
        food_list = random.sample(rain, 3)
    elif current_weather == 'Snow':
        food_list = random.sample(snow, 3)
    else:
        food_list = []
    return food_list

# 엑셀 파일 불러오기
excel_file = '음식메뉴_추가.xlsx'
df = pd.read_excel(excel_file)

# 사용자에게 구 이름 입력 받기
target_gu = input("구 이름을 입력하세요: ")

# 입력한 구 이름에 해당하는 데이터 필터링
filtered_df = df[df['구 이름'] == target_gu]

# 날씨 데이터 받아오기
current_weather = getNowCity("Seoul")

# 음식메뉴 컬럼 값과 일치하는 데이터 필터링
food_list = get_food_list(current_weather)
matching_df = filtered_df[filtered_df['음식메뉴'].str.contains('|'.join(food_list), case=False, na=False)]

# 나머지 데이터 중 별점을 기준으로 내림차순 정렬 후 상위 20개 선택
rest_df = filtered_df[~filtered_df.index.isin(matching_df.index)].sort_values(by='별점', ascending=False).head(20 - len(matching_df))

# 별점을 기준으로 내림차순 정렬 후 상위 20개 선택
top_rated_df = pd.concat([matching_df, rest_df]).sort_values(by='별점', ascending=False).head(30)

# 입력한 구의 중심 좌표 설정
center_lat, center_lon = top_rated_df['위도(y)'].mean(), top_rated_df['경도(x)'].mean()

# 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# 각 데이터행마다 지도에 마커 추가
for index, row in top_rated_df.iterrows():
    lat, lon = row['위도(y)'], row['경도(x)']
    popup_text = f"<div style='font-size: 12px;'>상호명: {row['상호명']}<br>도로명주소: {row['도로명주소']}<br>카테고리: {row['카테고리']}<br>사이트: <a href='{row['사이트']}' target='_blank'>{row['사이트']}</a><br>별점: {row['별점']}</div>"
    popup = folium.Popup(IFrame(popup_text, width=300, height=140), parse_html=True)

    # 별점에 따라 마커 색상 설정
    if row['별점'] == 5:
        marker_color = 'red'
    elif row['별점'] >= 4.7:
        marker_color = 'orange'
    elif row['별점'] >= 4.5:
        marker_color = 'blue'
    else:
        marker_color = 'black'

    marker = folium.Marker(location=[lat, lon], popup=popup, icon=folium.Icon(color=marker_color))
    marker.add_to(m)

# 지도를 창으로 띄우기
display(m)

In [79]:
import pandas as pd
import folium

# 엑셀 파일 불러오기
excel_file = '음식메뉴_추가.xlsx'
df = pd.read_excel(excel_file)

# 날씨 데이터 가져오기 (getNowCity 함수와 get_food_list 함수 호출 필요)
weather_main = getNowCity("Seoul")
food_list = get_food_list(weather_main)

# 음식메뉴 컬럼 값과 일치하는 데이터 필터링
matching_df = df[df['음식메뉴'].str.contains('|'.join(food_list), case=False, na=False)]

# matching_df가 비어있지 않은 경우에만 중심 좌표 설정
if not matching_df.empty:
    center_lat, center_lon = matching_df['위도(y)'].mean(), matching_df['경도(x)'].mean()
    
    # 지도 생성
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # 각 데이터행마다 지도에 마커 추가
    for index, row in matching_df.iterrows():
        lat, lon = row['위도(y)'], row['경도(x)']
        popup_text = f"상호명: {row['상호명']}<br>도로명주소: {row['도로명주소']}<br>카테고리: {row['카테고리']}<br>음식메뉴: {row['음식메뉴']}<br>사이트: <a href='{row['사이트']}' target='_blank'>{row['사이트']}</a><br>별점: {row['별점']}"
        popup = folium.Popup(popup_text, parse_html=True)
        marker = folium.Marker(location=[lat, lon], popup=popup)
        marker.add_to(m)

    # 지도를 창으로 띄우기
    m
else:
    print("매칭되는 데이터가 없습니다.")

TypeError: can only join an iterable